In [84]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Load dataset
First step is to load up the dataset to train the model with.
We're going to use the digits sample dataset from scikit-learn. This dataset contains handwritten digits along with the number that is represented in each sample. 

The input data is a 8x8 matrix, serialized as a single byte array. 
The label is a number representing the digit (0 - 9).

In [87]:
digit_dataset = load_digits(10)

X = digit_dataset.data
y = digit_dataset.target.reshape(-1,1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y)
y = y.todense()

## Create model
The model is a three layer dense neural network. We use the `Sequential` function to create a set of layers that are automatically connected. This makes the model much easier on the eyes.

In [111]:
import cntk as C
from cntk.layers import Sequential, Dense, For

In [113]:
model = Sequential([
    Dense(64), 
    Dense(32),
    Dense(10)
])

Composite(Dense): Placeholder('keep', [???], [???]) -> Output('Block3163_Output_0', [???], [???])

To actually train the model you need to define the mathmatical function for the model, by binding an input variable.
Since CNTK has a functional API, you simply call the model with the input variable as input.

In [96]:
features = C.input_variable(64)

z = model(features)

## Define the objective for the model
The objective is a categorical cross entropy loss function. A value close to zero indicates that the correct category was found for the handwritten digit. A high value indicates that we're far away from the right answer.

We also define an error function to measure performance later on.

In [97]:
label = C.input_variable(10)

loss = C.cross_entropy_with_softmax(z, label)
label_error = C.classification_error(z, label)

## Define the training procedure
To train the model we use a Stogastic Gradient Descent function. The trainer instance we create here is going to coordinate the training process using SGD and our loss function.

In [101]:
learning_rate = 0.1
learning_rate_schedule = C.learning_parameter_schedule(learning_rate)

learner = C.sgd(z.parameters, learning_rate_schedule)

trainer = C.Trainer(z, (loss, label_error), [learner])

## Train the model
To train the model we split the dataset into two pieces, a training set and a validation set.
We then use the training set to train the model in 10 epochs with minibatches of 64 samples.

In [102]:
minibatch_size = 64

train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2)

for _epoch in range(1, 10):
    for batch_offset in range(0, train_X.shape[0], minibatch_size):
        features_batch = train_X[batch_offset:batch_offset + minibatch_size]
        labels_batch = train_y[batch_offset:batch_offset + minibatch_size]
        
        input_map = {
            features: features_batch,
            label: labels_batch
        }

        trainer.train_minibatch(input_map)

C:\Anaconda3\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1743") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
C:\Anaconda3\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1744") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


## Validate the model
When training is completed, we can validate the model with a similar procedure as in the previous block.
The final score that is returned by `test_minibatch` is the final score for the model.

In [114]:
for batch_offset in range(0, len(test_X), minibatch_size):
    features_batch = test_X[batch_offset:batch_offset + minibatch_size]
    labels_batch = test_y[batch_offset:batch_offset + minibatch_size]
        
    input_map = {
        features: features_batch,
        label: labels_batch
    }
        
    score = trainer.test_minibatch(input_map)    
    
    print("Model score: {}".format(score))

Model score: 0.84375
Model score: 0.890625
Model score: 0.875
Model score: 0.859375
Model score: 0.875
Model score: 0.925


C:\Anaconda3\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1743") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
C:\Anaconda3\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input1744") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


In [108]:
print(score)

0.925


## Store the model
The final step is to store the model in ONNX format so we can import it in Java later on.

In [109]:
z.save('model.onnx', cntk.ModelFormat.ONNX)